In [1]:
import cmocean 
import cosima_cookbook as cc
import dask.distributed as dsk
import gsw
import numpy as np
import xarray as xr

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

figdir = '/home/561/jn8053/g_e14/figures-tmp/easterlies-collaborative/'

clnt = dsk.Client()
clnt

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/38833/status,
Dashboard: /proxy/38833/status,Workers: 4
Total threads: 16,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39489,Workers: 4
Dashboard: /proxy/38833/status,Total threads: 16
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:41181,Total threads: 4
Dashboard: /proxy/33971/status,Memory: 62.50 GiB
Nanny: tcp://127.0.0.1:45709,


In [2]:
session_ryf = cc.database.create_session()
experiment_ryf = '01deg_jra55v13_ryf9091'
#session_eas = cc.database.create_session('/g/data/v45/akm157/model_data/access-om2/01deg_jra55v13_ryf9091_easterlies_up10/easterlies.db')
experiment_eup = '01deg_jra55v13_ryf9091_easterlies_up10'
experiment_edo = '01deg_jra55v13_ryf9091_easterlies_down10'

ti = '2150-01-01'
tf = '2160-12-31'

In [ ]:
from cosima_cookbook import explore
ee = explore.ExperimentExplorer(session = session_ryf, experiment = experiment_eup)
ee

In [3]:
ht = cc.querying.getvar(experiment_ryf, 'ht', session_ryf, n = 1).sel(yt_ocean = slice(None, -58))
iso = xr.open_dataset('~/g_e14/misc/Antarctic_isobath_1000m.nc')

In [4]:
temp = {}
salt = {}
temp['ctrl'] = cc.querying.getvar(experiment_ryf, 'temp', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))-273.15
temp['up'] = cc.querying.getvar(experiment_eup, 'temp', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))-273.15
temp['down'] = cc.querying.getvar(experiment_edo, 'temp', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))-273.15
salt['ctrl'] = cc.querying.getvar(experiment_ryf, 'salt', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))
salt['up'] = cc.querying.getvar(experiment_eup, 'salt', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))
salt['down'] = cc.querying.getvar(experiment_edo, 'salt', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))

In [5]:
lat_slice = slice(-80,-58)
lon_slice = slice(-280, 80)

def shelf_mask_isobath(var):
    '''
    Masks ACCESS-OM2-01 variables by the region polewards of the 1000m isobath as computed using 
    a script contributed by Adele Morrison.
    Only to be used with ACCESS-OM2-0.1 output!
    '''
    contour_file = np.load('/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz')
    shelf_mask = contour_file['contour_masked_above']
    yt_ocean = contour_file['yt_ocean']
    xt_ocean = contour_file['xt_ocean']
    # Mask values that are non-zero
    shelf_mask[np.where(shelf_mask!=0)] = np.nan
    shelf_mask = shelf_mask+1
    shelf_map = np.nan_to_num(shelf_mask)
    shelf_mask = xr.DataArray(shelf_mask, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    shelf_map = xr.DataArray(shelf_map, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    # Multiply the variable with the mask, we need to account for the shape of the mask. 
    # The mask uses a northern cutoff of 59S.
    masked_var = var.sel(yt_ocean = lat_slice) * shelf_mask
    return masked_var, shelf_map

In [6]:
ht_masked, shelf_mask = shelf_mask_isobath(ht)

In [7]:
# Area for weighting
area_t = cc.querying.getvar(experiment_ryf, 'area_t', session_ryf, ncfile = 'ocean_grid.nc', n = -1)
area_t = area_t.sel(yt_ocean = slice(None, -58)).sel(xt_ocean = lon_slice)
area_t = area_t * shelf_mask.sel(yt_ocean = lat_slice).sel(xt_ocean = lon_slice)
area_sum = area_t.sum('xt_ocean').sum('yt_ocean')

In [8]:
area_sum.values

array(4.843247e+12, dtype=float32)

In [9]:
(1.5e6*1e-2)/4.8e12

3.125e-09

### Upper 100m contribution

In [ ]:
temp_surf_ctrl = temp['ctrl'].sel(st_ocean = slice(0, 100))
salt_surf_ctrl = salt['ctrl'].sel(st_ocean = slice(0, 100))

p_ctrl = gsw.p_from_z(-temp_surf_ctrl['st_ocean'], temp_surf_ctrl['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_surf_ctrl, p_ctrl, salt_surf_ctrl['xt_ocean'], salt_surf_ctrl['yt_ocean'])

ρ_mean_ctrl = gsw.rho(salt_surf_ctrl.mean('time'), temp_surf_ctrl.mean('time'), p_ctrl)
ρ_anom_ctrl = (gsw.rho(salt_surf_ctrl, temp_surf_ctrl, p_ctrl) - ρ_mean_ctrl) / ρ_mean_ctrl

# Height of t-cells
dzt = cc.querying.getvar(experiment_ryf, 'dzt', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_surf_ctrl = (ρ_anom_ctrl * dzt.sel(st_ocean = slice(0, 100))).sum('st_ocean')
slev_surf_ctrl = slev_surf_ctrl.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_surf_ctrl = slev_surf_ctrl.load()

# Averaging on shelf
slev_surf_ctrl_shelf, shelf_mask = shelf_mask_isobath(slev_surf_ctrl)
slev_surf_ctrl_shelf = (slev_surf_ctrl_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_surf_ctrl_shelf.to_netcdf('slev_surf_ctrl_shelf.nc')

In [ ]:
clnt.restart()

In [ ]:
temp_surf_up = temp['up'].sel(st_ocean = slice(0, 100))
salt_surf_up = salt['up'].sel(st_ocean = slice(0, 100))

p_up = gsw.p_from_z(-temp_surf_up['st_ocean'], temp_surf_up['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_surf_up, p_up, salt_surf_up['xt_ocean'], salt_surf_up['yt_ocean'])

ρ_mean_up = gsw.rho(salt_surf_up.mean('time'), temp_surf_up.mean('time'), p_up)
ρ_anom_up = (gsw.rho(salt_surf_up, temp_surf_up, p_up) - ρ_mean_up) / ρ_mean_up

# Height of t-cells
dzt = cc.querying.getvar(experiment_eup, 'dzt', session_eas, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_surf_up = (ρ_anom_up * dzt.sel(st_ocean = slice(0, 100))).sum('st_ocean')
slev_surf_up = slev_surf_up.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_surf_up = slev_surf_up.load()

# Averaging on shelf
slev_surf_up_shelf, shelf_mask = shelf_mask_isobath(slev_surf_up)
slev_surf_up_shelf = (slev_surf_up_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_surf_up_shelf.to_netcdf('slev_surf_up_shelf.nc')

In [ ]:
clnt.restart()

In [ ]:
temp_surf_down = temp['down'].sel(st_ocean = slice(0, 100))
salt_surf_down = salt['down'].sel(st_ocean = slice(0, 100))

p_down = gsw.p_from_z(-temp_surf_down['st_ocean'], temp_surf_down['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_surf_down, p_down, salt_surf_down['xt_ocean'], salt_surf_down['yt_ocean'])

ρ_mean_down = gsw.rho(salt_surf_down.mean('time'), temp_surf_down.mean('time'), p_down)
ρ_anom_down = (gsw.rho(salt_surf_down, temp_surf_down, p_down) - ρ_mean_down) / ρ_mean_down

# Height of t-cells
dzt = cc.querying.getvar(experiment_edo, 'dzt', session_eas, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_surf_down = (ρ_anom_down * dzt.sel(st_ocean = slice(0, 100))).sum('st_ocean')
slev_surf_down = slev_surf_down.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_surf_down = slev_surf_down.load()

# Averaging on shelf
slev_surf_down_shelf, shelf_mask = shelf_mask_isobath(slev_surf_down)
slev_surf_down_shelf = (slev_surf_down_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_surf_down_shelf.to_netcdf('slev_surf_down_shelf.nc')

In [ ]:
clnt.restart()

### Bottom 100m

In [ ]:
depth_array = temp['ctrl'] * 0 + temp['ctrl'].st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)

temp_bott_ctrl = temp['ctrl'].where(depth_array['st_ocean'] >= max_depth-100)
salt_bott_ctrl = salt['ctrl'].where(depth_array['st_ocean'] >= max_depth-100)

p_ctrl = gsw.p_from_z(-temp_bott_ctrl['st_ocean'], temp_bott_ctrl['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_bott_ctrl, p_ctrl, salt_bott_ctrl['xt_ocean'], salt_bott_ctrl['yt_ocean'])

# Density changes due to temp/salt changes
ρ_mean_ctrl = gsw.rho(salt_bott_ctrl.mean('time'), temp_bott_ctrl.mean('time'), p_ctrl)
ρ_anom_ctrl = (gsw.rho(salt_bott_ctrl, temp_bott_ctrl, p_ctrl) - ρ_mean_ctrl) / ρ_mean_ctrl

# Height of t-cells
dzt = cc.querying.getvar(experiment_ryf, 'dzt', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_bott_ctrl = (ρ_anom_ctrl * dzt.where(depth_array['st_ocean'] >= max_depth-100)).sum('st_ocean')
slev_bott_ctrl = slev_bott_ctrl.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})

# Averaging on shelf
slev_bott_ctrl_shelf, shelf_mask = shelf_mask_isobath(slev_bott_ctrl)
slev_bott_ctrl_shelf = (slev_bott_ctrl_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum
slev_bott_ctrl_shelf = slev_bott_ctrl_shelf.load()

slev_bott_ctrl_shelf.to_netcdf('slev_bott_ctrl_shelf.nc')

In [ ]:
clnt.restart()

In [ ]:
depth_array = temp['up'] * 0 + temp['up'].st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)

depth_array = temp['up'] * 0 + temp['up'].st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)

temp_bott_up = temp['up'].where(depth_array.st_ocean >= max_depth-100)
salt_bott_up = salt['up'].where(depth_array.st_ocean >= max_depth-100)

p_up = gsw.p_from_z(-temp_bott_up['st_ocean'], temp_bott_up['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_bott_up, p_up, salt_bott_up['xt_ocean'], salt_bott_up['yt_ocean'])

# Density changes due to temp/salt changes
ρ_mean_up = gsw.rho(salt_bott_up.mean('time'), temp_bott_up.mean('time'), p_up)
ρ_anom_up = (gsw.rho(salt_bott_up, temp_bott_up, p_up) - ρ_mean_up) / ρ_mean_up

# Height of t-cells
dzt = cc.querying.getvar(experiment_eup, 'dzt', session_eas, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_bott_up = (ρ_anom_up * dzt.where(depth_array.st_ocean >= max_depth-100)).sum('st_ocean')
slev_bott_up = slev_bott_up.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_bott_up = slev_bott_up.load()

# Averaging on shelf
slev_bott_up_shelf, shelf_mask = shelf_mask_isobath(slev_bott_up)
slev_bott_up_shelf = (slev_bott_up_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_bott_up_shelf.to_netcdf('slev_bott_up_shelf.nc')

In [ ]:
clnt.restart()

In [ ]:
depth_array = temp['down'] * 0 + temp['down'].st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)

depth_array = temp['down'] * 0 + temp['down'].st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)

temp_bott_down = temp['down'].where(depth_array.st_ocean >= max_depth-100)
salt_bott_down = salt['down'].where(depth_array.st_ocean >= max_depth-100)

p_down = gsw.p_from_z(-temp_bott_down['st_ocean'], temp_bott_down['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_bott_down, p_down, salt_bott_down['xt_ocean'], salt_bott_down['yt_ocean'])

ρ_mean_down = gsw.rho(salt_bott_down.mean('time'), temp_bott_down.mean('time'), p_down)
ρ_anom_down = (gsw.rho(salt_bott_down, temp_bott_down, p_down) - ρ_mean_down) / ρ_mean_down

# Height of t-cells
dzt = cc.querying.getvar(experiment_edo, 'dzt', session_eas, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_bott_down = (ρ_anom_down * dzt.where(depth_array.st_ocean >= max_depth-100)).sum('st_ocean')
slev_bott_down = slev_bott_down.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_bott_down = slev_bott_down.load()

# Averaging on shelf
slev_bott_down_shelf, shelf_mask = shelf_mask_isobath(slev_bott_down)
slev_bott_down_shelf = (slev_bott_down_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_bott_down_shelf.to_netcdf('slev_bott_down_shelf.nc')

# Steric sea level changes including the whole water column

$$
-\int \frac{\rho - \rho_{0}}{\rho_{0}} dz
$$

with $\rho_{0} = 1035$. Both density and cell height are taken from the same experiment

In [10]:
temp_bott_ctrl = temp['ctrl']
salt_bott_ctrl = salt['ctrl']

p_ctrl = gsw.p_from_z(-temp_bott_ctrl['st_ocean'], temp_bott_ctrl['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_bott_ctrl, p_ctrl, salt_bott_ctrl['xt_ocean'], salt_bott_ctrl['yt_ocean'])

# Density changes due to temp/salt changes
ρ_anom_ctrl = (gsw.rho(salt_bott_ctrl, temp_bott_ctrl, p_ctrl) - 1035) / 1035

# Height of t-cells
dzt = cc.querying.getvar(experiment_ryf, 'dzt', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_bott_ctrl = (ρ_anom_ctrl * dzt).sum('st_ocean')
slev_bott_ctrl = slev_bott_ctrl.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})

# Averaging on shelf
slev_bott_ctrl_shelf, shelf_mask = shelf_mask_isobath(slev_bott_ctrl)
slev_bott_ctrl_shelf = (slev_bott_ctrl_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum
slev_bott_ctrl_shelf = slev_bott_ctrl_shelf.load()

slev_bott_ctrl_shelf.to_netcdf('slev_steric_onshelf_control1.nc')

2022-08-18 16:52:59,256 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2022-08-18 16:53:01,190 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2022-08-18 16:53:21,030 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2022-08-18 16:54:36,954 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-08-18 16:55:28,919 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [9]:
clnt.restart()

distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/34135/status,
Dashboard: /proxy/34135/status,Workers: 4
Total threads: 16,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46475,Workers: 4
Dashboard: /proxy/34135/status,Total threads: 16
Started: 15 minutes ago,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:46577,Total threads: 4
Dashboard: /proxy/46457/status,Memory: 62.50 GiB
Nanny: tcp://127.0.0.1:40357,


In [10]:
temp_bott_up = temp['up']
salt_bott_up = salt['up']

p_up = gsw.p_from_z(-temp_bott_up['st_ocean'], temp_bott_up['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_bott_up, p_up, salt_bott_up['xt_ocean'], salt_bott_up['yt_ocean'])

# Density changes due to temp/salt changes
ρ_anom_up = (gsw.rho(salt_bott_up, temp_bott_up, p_up) - 1035) / 1035

# Height of t-cells
dzt = cc.querying.getvar(experiment_eup, 'dzt', session_eas, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_bott_up = (ρ_anom_up * dzt).sum('st_ocean')
slev_bott_up = slev_bott_up.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_bott_up = slev_bott_up.load()

# Averaging on shelf
slev_bott_up_shelf, shelf_mask = shelf_mask_isobath(slev_bott_up)
slev_bott_up_shelf = (slev_bott_up_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_bott_up_shelf.to_netcdf('slev_steric_onshelf_up.nc')

In [11]:
clnt.restart()

distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/34135/status,
Dashboard: /proxy/34135/status,Workers: 4
Total threads: 16,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46475,Workers: 4
Dashboard: /proxy/34135/status,Total threads: 16
Started: 32 minutes ago,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:35329,Total threads: 4
Dashboard: /proxy/46595/status,Memory: 62.50 GiB
Nanny: tcp://127.0.0.1:40357,


In [12]:
temp_bott_down = temp['down']
salt_bott_down = salt['down']

p_down = gsw.p_from_z(-temp_bott_down['st_ocean'], temp_bott_down['yt_ocean'])
salt_abs = gsw.SA_from_SP(salt_bott_down, p_down, salt_bott_down['xt_ocean'], salt_bott_down['yt_ocean'])

ρ_anom_down = (gsw.rho(salt_bott_down, temp_bott_down, p_down) - 1035) / 1035

# Height of t-cells
dzt = cc.querying.getvar(experiment_edo, 'dzt', session_eas, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

# Integrating surface layer
slev_bott_down = (ρ_anom_down * dzt).sum('st_ocean')
slev_bott_down = slev_bott_down.chunk({'xt_ocean':'500MB', 'yt_ocean':'500MB'})
slev_bott_down = slev_bott_down.load()

# Averaging on shelf
slev_bott_down_shelf, shelf_mask = shelf_mask_isobath(slev_bott_down)
slev_bott_down_shelf = (slev_bott_down_shelf*area_t).sum('xt_ocean').sum('yt_ocean') / area_sum

slev_bott_down_shelf.to_netcdf('slev_steric_onshelf_down.nc')

In [13]:
clnt.restart()

distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/34135/status,
Dashboard: /proxy/34135/status,Workers: 4
Total threads: 16,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46475,Workers: 4
Dashboard: /proxy/34135/status,Total threads: 16
Started: 1 hour ago,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:46189,Total threads: 4
Dashboard: /proxy/45221/status,Memory: 62.50 GiB
Nanny: tcp://127.0.0.1:40357,


# Bottom pressure term

$$
\frac{p_{bot}}{g \rho} 
$$

and `pbot_t` from the model is in dbar, so to convert to `N/m2`, multiply by 1e4. 

In [9]:
pbot = {}
pbot['ctrl'] = cc.querying.getvar(experiment_ryf, 'pbot_t', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))
pbot['up'] = cc.querying.getvar(experiment_eup, 'pbot_t', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))
pbot['down'] = cc.querying.getvar(experiment_edo, 'pbot_t', session_ryf, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))

In [11]:
pbot_ctrl_shelf, shelf_mask = shelf_mask_isobath(pbot['ctrl'])
pbot_ctrl_shelf = (1/(9.8*1035))*(pbot_ctrl_shelf*1e4).mean(['xt_ocean', 'yt_ocean'])
pbot_ctrl_shelf = pbot_ctrl_shelf.load()

pbot_ctrl_shelf.to_netcdf('pbot_onshelf_control.nc')

In [12]:
pbot_up_shelf, shelf_mask = shelf_mask_isobath(pbot['up'])
pbot_up_shelf = (1/(9.8*1035))*(pbot_up_shelf*1e4).mean(['xt_ocean', 'yt_ocean'])
pbot_up_shelf = pbot_up_shelf.load()

pbot_up_shelf.to_netcdf('pbot_onshelf_up.nc')

In [13]:
pbot_down_shelf, shelf_mask = shelf_mask_isobath(pbot['down'])
pbot_down_shelf = (1/(9.8*1035))*(pbot_down_shelf*1e4).mean(['xt_ocean', 'yt_ocean'])
pbot_down_shelf = pbot_down_shelf.load()

pbot_down_shelf.to_netcdf('pbot_onshelf_down.nc')